<a href="https://colab.research.google.com/github/Sweta-Das/DLCourse-LLMUsingSemanticSearch/blob/main/LLM_Keyword_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cohere
!pip install weaviate-client

In [ ]:
pip install python-dotenv

In [ ]:
# Loading database and necessary API keys
import os
from dotenv import load_dotenv, find_dotenv

# reading local .env file
_ = load_dotenv(find_dotenv())

In [ ]:
# Importing weaviate
import weaviate
auth_config = weaviate.auth.AuthApiKey(api_key= "76320a90-53d8-42bc-b41d-678647c6672e")

Weaviate is an open-source database that allows to connect to an online database. Here, to access the Wikipedia database.<br/>
It has keyword search capabilities & also vector-search capabilities.

In [ ]:
client = weaviate.Client(
    url = "https://cohere-demo.weaviate.network/",
    auth_client_secret = auth_config,
    additional_headers = {"X-Cohere-Api-Key": "nwgNN3UCHd2g0wEzM0pYQd4KSxoRVJkLPmB9d6AP"}
)

The given url points to an online database that contains 10 million records. Each row of this records are a paragraph from Wikipedia. These 10 million records are in 10 different languages (1 million each).

In [ ]:
client.is_ready()

True

### Keyword Search

For keyword search we're trying to access articles and providing certain properties of articles as arguments.

In [ ]:
def keyword_search(query, results_lang= 'en', properties= ["title", "url", "text"], num_results= 3):

  # Filter used to limiting text to only English results
  where_filter = {
      "path": ["lang"],
      "operator": "Equal",
      "valueString": results_lang
  }

  response = (client.query.get("Articles", properties)
              .with_bm25(
                  query= query
              )
              .with_where(where_filter)
              .with_limit(num_results) # number of results
              .do()
            )

  result = response['data']['Get']['Articles']
  return result

bm25 is a commonly used keyword (lexical) search algo. that scores the doc in the archive vs. the query.

In [ ]:
query = "What is the most viewed televised event?"
keyword_search_results = keyword_search(query)
print(keyword_search_results)

[{'text': 'The most active Gamergate supporters or "Gamergaters" said that Gamergate was a movement for ethics in games journalism, for protecting the "gamer" identity, and for opposing "political correctness" in video games and that any harassment of women was done by others not affiliated with Gamergate. They argued that the close relationships between journalists and developers demonstrated a conspiracy among reviewers to focus on progressive social issues. Some supporters pointed to what they considered disproportionate praise for games such as "Depression Quest" and "Gone Home", which feature unconventional gameplay and stories with social implications, while they viewed traditional AAA games as downplayed. False claims of the "ethics in game journalism" had started as early as 2012, when Geoff Keighley was accused of such unethical behavior when he was presenting information about "Halo 4" among advertisements for Mountain Dew and Doritos, an event called "Doritosgate" in the gam

Checking output in other languages (fr, it, ja, hi)

In [ ]:
def keyword_search(query, results_lang= 'hi', properties= ["title", "url", "text"], num_results= 3):

  # Filter used to limiting text to only English results
  where_filter = {
      "path": ["lang"],
      "operator": "Equal",
      "valueString": results_lang
  }

  response = (client.query.get("Articles", properties)
              .with_bm25(
                  query= query
              )
              .with_where(where_filter)
              .with_limit(num_results) # number of results
              .do()
            )

  result = response['data']['Get']['Articles']
  return result

In [ ]:
query = "What is the most viewed televised event?"
keyword_search_results = keyword_search(query)
print(keyword_search_results)

[{'text': 'फरवरी 2020 में, विश्व स्वास्थ्य संगठन (WHO) ने कहा कि कोविड-१९ का टीका 18 महीने से कम समय में उपलब्ध होने की उम्मीद नहीं की जा सकती। किसी नई महामारी के टीके की तैयारी के नवाचारों का गठबंधन (CEPI) (The ) जो वैक्सीन उम्मीदवारों के तेजी से विकास के लिए दुनिया भर में US$2 बिलियन फंड आयोजित कर रहा है अप्रैल में संकेत दिया गया कि 12 महीने से कम या 2021 की शुरुआत में आपातकालीन उपयोग के लिए इसका टीका उपलब्ध हो सकता है। 4 मई 2020 को WHO ने COVID-19 संक्रमण को रोकने के लिए और टीकों के तेजी से विकास का समर्थन करने के लिए चालीस देशों से US$8 बिलियन जुटाने के लिए एक टेलीथॉन (televised fundraising event )का आयोजन किया, साथ ही कई वैक्सीन उम्मीदवारों के एक साथ मूल्यांकन के लिए चरण II-III नैदानिक परीक्षण के नाम से एक अंतर्राष्ट्रीय "सॉलिडैरिटी ट्रायल" की तैनाती की घोषणा की, ।', 'title': 'कोविड-19 का टीका', 'url': 'https://hi.wikipedia.org/wiki?curid=1174739'}, {'text': 'Meanwhile, James Boswell, then in Paris, offered to escort Thérèse to Rousseau. (Boswell had earlier met Rousseau and Thérè

Output is in Hindi.

### Trying to modify the search options

In [ ]:
properties = ["text", "title", "url", "views", "lang"]

In [ ]:
def print_results(result):

  # Printing results with colorful formatting
  for i, item in enumerate(result):
    print(f'item {i}')
    for key in item.keys():
      print(f"{key}:{item.get(key)}")
      print()
    print()

In [ ]:
print_results(keyword_search_results)

item 0
text:फरवरी 2020 में, विश्व स्वास्थ्य संगठन (WHO) ने कहा कि कोविड-१९ का टीका 18 महीने से कम समय में उपलब्ध होने की उम्मीद नहीं की जा सकती। किसी नई महामारी के टीके की तैयारी के नवाचारों का गठबंधन (CEPI) (The ) जो वैक्सीन उम्मीदवारों के तेजी से विकास के लिए दुनिया भर में US$2 बिलियन फंड आयोजित कर रहा है अप्रैल में संकेत दिया गया कि 12 महीने से कम या 2021 की शुरुआत में आपातकालीन उपयोग के लिए इसका टीका उपलब्ध हो सकता है। 4 मई 2020 को WHO ने COVID-19 संक्रमण को रोकने के लिए और टीकों के तेजी से विकास का समर्थन करने के लिए चालीस देशों से US$8 बिलियन जुटाने के लिए एक टेलीथॉन (televised fundraising event )का आयोजन किया, साथ ही कई वैक्सीन उम्मीदवारों के एक साथ मूल्यांकन के लिए चरण II-III नैदानिक परीक्षण के नाम से एक अंतर्राष्ट्रीय "सॉलिडैरिटी ट्रायल" की तैनाती की घोषणा की, ।

title:कोविड-19 का टीका

url:https://hi.wikipedia.org/wiki?curid=1174739


item 1
text:Meanwhile, James Boswell, then in Paris, offered to escort Thérèse to Rousseau. (Boswell had earlier met Rousseau and Thérèse at Mo